In [7]:
import numpy as np
from IPython.display import HTML, display, clear_output, update_display
import ipywidgets as widgets
from itertools import product
from heapq import heappop, heappush
from collections import deque
import json
from copy import deepcopy, copy
from functools import lru_cache
import itertools as it
import time
import os

class Board():
    def __init__(self, INIT_STATE):
        self.squares = INIT_STATE
        self.start_board = copy(INIT_STATE)
        self.rows = INIT_STATE.shape[0]
        self.cols = INIT_STATE.shape[1]
        r, c = np.where(INIT_STATE=="A")
        self.own_pos = (r[0],c[0])
        self.state = self.own_pos
        self.start_board[self.own_pos] = "f"
        rg, cg = np.where(INIT_STATE=="B")
        self.goal_pos = (rg[0],cg[0])
        self.cost = {"w":100, "m":50, "f":10, "g":5, "r":1}
        self.color = {"A":"red", "B":"green", ".":"white", "#":"darkgrey", "w":"blue", "m":"grey", "f":"darkgreen", "g":"lightgreen", "r":"brown","W":"blue", "M":"grey", "F":"darkgreen", "G":"lightgreen", "R":"brown", "1":"blue", "2":"grey", "3":"darkgreen", "4":"lightgreen", "5":"brown", "6":"blue", "7":"grey", "8":"darkgreen", "9":"lightgreen", "0":"brown"}
        self.textmap = {"Ax":"A", "Bx":"B", "B*":"B", "B+":"B", "wx":"W", "mx":"M","fx":"F","gx":"G","rx":"R", "w*":"1", "m*":"2", "f*":"3", "g*":"4", "r*":"5","w+":"6", "m+":"7", "f+":"8", "g+":"9", "r+":"0"}
        self.text = {".":"", "#":"", "w":"", "m":"", "f":"", "g":"", "r":"","W":"x","M":"x", "F":"x", "G":"x", "R": "x","1":"*", "2":"*","3":"*", "4":"*", "5":"*","6":"+", "7":"+","8":"+", "9":"+", "0":"+" }
        display(HTML(self.board_html()), display_id=str(id(self)))
        
    def move(self, move):
        old_x, old_y = self.own_pos  
        if move==0: #right
            return (old_x+1,old_y)
        elif move==1: #up
            return (old_x,old_y-1)
        elif move==2: #left
            return (old_x-1,old_y)
        elif move==3: #down
            return (old_x,old_y+1)
    
    def set_state(self, state, init=False):
        self.squares = copy(self.start_board)
        self.squares[state] = "A"
        self.own_pos = state
        self.state = state
        if init:
            self.squares[self.goal_pos] = "B"
        self.display_state()
    
    def add_marker(self, poslist, mark="x"):
        self.squares = copy(self.start_board)
        for pos in poslist:
            #print(self.squares[pos],mark)
            self.squares[pos] = self.textmap[self.squares[pos]+mark]
        self.display_state()
    
    def get_next_states(self, return_moves=False):
        # Function to get next possible moves OR next possible states, depending on the return_moves-flag. 
        moves = []
        for m in range(4):
            next_x, next_y = self.move(m)
            #print(n)
            if next_x in range(self.rows) and next_y in range(self.cols):
                next_val = self.squares[self.move(m)]
                if next_val != "#":
                    moves.append(m)
        next_states = [self.move(m) for m in moves]
        return next_states
    
    def display_state(self):
        update_display(HTML(self.board_html()), display_id=str(id(self)))
    
    def square_html(self, sq) -> str:
        # Represent each square in HTML
        size = 150
        if sq == "A": # If A, display A
            text = "A"
        elif sq == "B": # If B, display B
            text = "B"
        else:
            text = self.text[sq] # No text
        color = self.color[sq] # Look up the carcolor-dict to retrieve the color.
        style = "background-color:{}; font-size:{}%; width:30px; height:30px; text-align:center; padding:0px; border: 2px solid black;" # Style for the square
        return ('<td style="' + style + '">{}').format(color, size, text)
    
    def board_html(self) -> str:
        # Represent the whole board in HTML 
        squares = [self.square_html(sq) for sq in self.squares.flatten()] # Flatten the matrix, and get HTML-representation for each square.
        row = ('<tr>' + '{}' * self.cols) # Create HTML-tag for each column
        return ('<table>' +  row * self.rows + '</table>').format(*squares) # Create the HTML-table, with the square-representations inserted. 

    
    def _repr_html_(self) -> str:
        #Own representation in HTML.
        return self.board_html()
    
    def h_func(self, s):
        goal_x, goal_y = self.goal_pos
        x_dist = abs(s[0]-goal_x)
        y_dist = abs(s[1]-goal_y)
        return x_dist+y_dist # manhattan distance
    
    def get_cost(self, pos):
        in_next = self.squares[pos]
        if in_next in self.cost.keys():
            return self.cost[in_next]
        else:
            #print(pos)
            #print("Not in costdict")
            return 0

def a_star_search(b, mode="astar", display_mode=False):
    # b is the board object. Must implement h_func(state)-method, get_next_states(), set_state() and state-attribute
    if mode in ["dijkstra","astar"]:
        openset = [(b.h_func(b.state), b.state)]
    else:
        openset = deque([b.state])
    closedset = {b.state:0}
    parent = {b.state: ''}
    expanded = 0
    while len(openset) > 0:
        if mode in ["dijkstra", "astar"]:
            f, s = heappop(openset)
        else:
            s = openset.popleft()
        if display_mode:
            print("Expanding " + s)
            print("Cost of s: " + str(f))
        b.set_state(s)
        if b.h_func(s) == 0: # If cost from s to goal is 0.
            path_to_s = [s]
            b.display_state()
            while s != '':
                if display_mode:
                    print(s)
                path_to_s.append(parent[s])
                s = parent[s]
            print("Successfully found solution. Path to solution consists of "+str(len(path_to_s[1:])-1)+" nodes.")
            print("Expanded a total of "+str(expanded)+" nodes.")
            return list(reversed(path_to_s))[1:], openset, closedset
        succ = b.get_next_states()
        #print("Successors: " + str(len(succ)))
        expanded += 1
        for newnode in succ:                
            if display_mode:
                print("Child "+newnode+" has cost of "+ str(b.h_func(newnode)))
            curr_cost = closedset[s] + b.get_cost(newnode)
            if (newnode in closedset and curr_cost < closedset[newnode]) or (newnode not in closedset.keys()): 
                # If new node is not in closed set or has lower cost than previous cost for same node.
                # Then we update the cost, and push it to the open set for re-expansion to find new cost of children
                closedset[newnode] = curr_cost
                parent[newnode] = s
                if mode in ["dijkstra", "bfs"]:
                    g = 0
                elif mode=="astar":
                    g = b.h_func(newnode)
                if mode in ["dijkstra", "astar"]:
                    heappush(openset, (curr_cost+g, newnode))
                else:
                    openset.appendleft(newnode)
                #if display_mode:
                #print("Child " + newnode + " with cost"+ str((curr_cost+b.h_func(new_n.state)))+ "  is either not seen before or has lower cost than previous ")
                #    print("Pushed "+ newnode + " to openset")
                #    print("New openset: "+ str(openset))
            else:
                if display_mode:
                    print("Child " + newnode + " has higher cost than previously seen")
    return False

def get_board1(boardname):
    file = open("boards/"+boardname+".txt", "r")
    lines = file.read().split("\n")
    arr = []
    for l in [line for line in lines if line !=""]:
        values = np.array(list(l)) 
        arr.append(values)
    file.close()
    return np.vstack(arr)

In [8]:
init_state = get_board1("board-2-3")

In [9]:
b = Board(init_state)

In [10]:
sol, openset, closedset = a_star_search(b, mode="dijkstra")

Successfully found solution. Path to solution consists of 49 nodes.
Expanded a total of 320 nodes.


In [11]:
b.add_marker([o[1] for o in openset], "*")

In [12]:
b.add_marker(list(closedset.keys()), "x")

In [288]:
b.add_marker(sol, "+")

In [224]:
b.squares

array([['g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'w', 'w', 'w', 'g',
        'g', 'g', 'g', 'g', 'm', 'm', 'm', 'm', 'm', 'm', 'm', '2', 'm',
        'm', 'B', 'r', 'r', 'r', 'r', 'r', 'r', 'r', '2', 'm', 'm', 'm',
        'm'],
       ['g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'w', 'w', 'w', 'w',
        'g', 'g', 'g', 'g', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', '2',
        'm', 'm', '2', '2', '2', '2', '2', '2', 'r', '2', 'g', 'g', 'g',
        'g'],
       ['g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'w', 'w', 'w',
        'w', 'g', 'g', 'g', 'g', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm',
        '2', 'm', '2', '2', '2', '2', 'g', '4', 'r', '4', 'g', 'g', 'g',
        'g'],
       ['f', 'f', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'w', 'w',
        'w', 'w', 'g', 'g', 'g', 'g', 'm', 'm', 'm', 'm', 'm', 'm', 'm',
        'm', '2', 'g', 'g', 'g', 'g', 'g', 'g', 'r', '5', 'g', 'g', 'g',
        'g'],
       ['f', 'f', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 

In [117]:
import time
#b = Board(init_state)
for i, s in enumerate(sol):
    b.set_state(s, not i)
    time.sleep(0.2)